In [1]:
import numpy as np
import pandas as pd
import re
import os
os.chdir(r'C:\Users\jadva\OneDrive\Desktop\College\Period 4\Building and mining knowledge graphs\Project\Movie Recommender\Final_Datasets')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
#integrate the different datasets


prime = pd.read_csv(r'amazon_prime_titles.csv')
prime["Source"] = "Amazon Prime"

disney = pd.read_csv(r'disney_plus_titles.csv')
disney["Source"] = "Disney Plus"

hulu = pd.read_csv(r'hulu_titles.csv')
hulu["Source"] = "Hulu"

netflix = pd.read_csv(r'netflix_titles.csv')
netflix["Source"] = "Netflix"

prime = prime[prime["type"]=="Movie"]
prime.reset_index(drop = True,inplace = True)

disney = disney[disney["type"]=="Movie"]
disney.reset_index(drop = True,inplace = True)

hulu = hulu[hulu["type"]=="Movie"]
hulu.reset_index(drop = True,inplace = True)

netflix = netflix[netflix["type"]=="Movie"]
netflix.reset_index(drop = True,inplace = True)

integrated_ott_movies = pd.concat([prime,disney,hulu,netflix])

In [3]:
#clean and process the movielens dataset

movielens_movies = pd.read_csv(r'movies.csv')

movielens_movies["release_year"] = movielens_movies["title"].apply(lambda x: re.findall(r'\([0-9]+\)',x))

movielens_movies["release_year_clean"] = np.nan

for i in range(len(movielens_movies)):
    try:
        movielens_movies["release_year_clean"][i] = movielens_movies["release_year"][i][-1]
        movielens_movies["release_year_clean"][i] = movielens_movies["release_year_clean"][i].replace("(","").replace(")","")
        movielens_movies["release_year_clean"][i] = int(movielens_movies["release_year_clean"][i])
    except:
        continue

movielens_movies = movielens_movies[movielens_movies["release_year_clean"].notnull()] #removing the rows with no year for the movie  
movielens_movies["title"] = movielens_movies["title"].apply(lambda x: re.sub(r'\([0-9]+\)','',x)) #removing year from title
movielens_movies["release_year"] = movielens_movies["release_year_clean"]
movielens_movies = movielens_movies.drop(columns=["release_year_clean"])
movielens_movies = movielens_movies[(movielens_movies["release_year"] >= integrated_ott_movies["release_year"].min()) & (movielens_movies["release_year"] <= 2017)]
movielens_movies.reset_index(drop = True,inplace = True)
movielens_movies['title'] = movielens_movies['title'].apply(lambda x:x.strip())


#filter based on common years in both
integrated_ott_movies = integrated_ott_movies[integrated_ott_movies["release_year"] <= 2017]
integrated_ott_movies.reset_index(drop = True,inplace = True)
integrated_ott_movies['title'] = integrated_ott_movies['title'].apply(lambda x:x.strip())
#take a subset of the data
integrated_ott_movies = integrated_ott_movies[['title','director','cast','country','rating','listed_in','Source','release_year']]

C:\Users\jadva\AppData\Local\Temp/ipykernel_2752/2256283742.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movielens_movies["release_year_clean"][i] = movielens_movies["release_year"][i][-1]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# now we match some titles considering release year also. we will use fuzzy string matching for the rest

In [4]:
#matched with titles and year

cond1 = integrated_ott_movies['title'].isin(movielens_movies['title'])
cond2 = integrated_ott_movies['release_year'].isin(movielens_movies['release_year'])

direct_match = movielens_movies.iloc[movielens_movies[cond1 & cond2].index]
direct_match.reset_index(drop = True,inplace = True)

not_matched = movielens_movies.drop(movielens_movies[cond1 & cond2].index)
not_matched.reset_index(drop = True,inplace = True)

not_matched = not_matched.drop_duplicates(subset = ['title','release_year'])
direct_match = direct_match.drop_duplicates(subset = ['title','release_year'])



C:\Users\jadva\AppData\Local\Temp/ipykernel_2752/1852564502.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  direct_match = movielens_movies.iloc[movielens_movies[cond1 & cond2].index]
C:\Users\jadva\AppData\Local\Temp/ipykernel_2752/1852564502.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_matched = movielens_movies.drop(movielens_movies[cond1 & cond2].index)


In [5]:
#matched with fuzzy matching taking a high degree of confidence
from rapidfuzz import process, fuzz
choices = integrated_ott_movies['title'].tolist()
not_matched['Matches'] = np.nan
not_matched_new_df = pd.DataFrame()


years = not_matched['release_year'].unique().tolist()
for year in years:
  choices = integrated_ott_movies[integrated_ott_movies['release_year'] == year]['title'].tolist()
  if(choices == []):
    continue
  else:
    df = not_matched[not_matched['release_year'] == year]
    df.reset_index(drop = True,inplace = True)
    for i in range(len(df)):
        df['Matches'][i] = process.extract(df['title'][i], choices, scorer=fuzz.WRatio, limit=1)
    not_matched_new_df = pd.concat([not_matched_new_df,df])
    
not_matched = not_matched_new_df.copy() 
not_matched.reset_index(drop = True,inplace = True)

not_matched["Scores"] = np.nan
not_matched["Scores"] = not_matched["Matches"].apply(lambda x: x[0][1]) 
not_matched["Matches"] = not_matched["Matches"].apply(lambda x: x[0][0]) 



not_matched_high_conf = not_matched[not_matched["Scores"] >= 95]
not_matched_low_conf = not_matched[(not_matched["Scores"] >= 90) & (not_matched["Scores"] < 95)]


not_matched_low_conf.reset_index(drop = True,inplace = True)
not_matched_low_conf['Scores_1'] = np.nan
not_matched_low_conf['Scores_2'] = np.nan
for i in range(len(not_matched_low_conf)):
    not_matched_low_conf['Scores_1'][i] = fuzz.token_set_ratio(not_matched_low_conf['Matches'][i], not_matched_low_conf['title'][i])
    not_matched_low_conf['Scores_2'][i] = fuzz.partial_ratio(not_matched_low_conf['Matches'][i], not_matched_low_conf['title'][i])
not_matched_low_conf = not_matched_low_conf[(not_matched_low_conf['Scores_1'] >= 95) & (not_matched_low_conf['Scores_2'] >= 95)]    


not_matched = pd.concat([not_matched_low_conf,not_matched_high_conf])
not_matched["Matches"] = not_matched["title"] 
# not_matched.drop(columns = ["Matches"],inplace = True)
not_matched.drop(columns = ["Matches","Scores","Scores_1","Scores_2"],inplace = True)

movielens_clean = pd.concat([direct_match,not_matched])
movielens_clean.reset_index(drop = True,inplace = True)
movielens_clean['title'] = movielens_clean['title'].apply(lambda x:x.strip()) 

C:\Users\jadva\AppData\Local\Temp/ipykernel_2752/1462143119.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Matches'][i] = process.extract(df['title'][i], choices, scorer=fuzz.WRatio, limit=1)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\jadva\AppData\Local\Temp/ipykernel_2752/1462143119.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [6]:
integrated_ott_movies = integrated_ott_movies.merge(movielens_clean,how = 'inner',on=['title','release_year'])
# integrated_ott_movies.to_excel(r'integrated_ott_movies.xlsx',index = False)

# processing the Movielens dataset

In [7]:
movie_review = pd.read_csv(r'ratings.csv')
movie_review.drop(columns = ['timestamp'],inplace = True)

unique_movie_ids = integrated_ott_movies['movieId'].unique().tolist()
movie_review = movie_review[movie_review['movieId'].isin(unique_movie_ids)]

movie_review.reset_index(drop = True, inplace = True)

In [8]:
import rdflib
from rdflib  import Graph, Literal, RDF, URIRef, Namespace,XSD,FOAF,OWL

In [9]:
def title_clean(x):
    x = str(x)
    x =  x.replace(" ","_")
    x = x.replace("-","_")
    x =  x.replace(r'"','')
    x = x.replace(" ","%20")
    x = x.replace('"',"%22")
    
    return x

# Knowledge graph 1 - With just OTT data

In [10]:

#Existing Schemas
SCHEMA = Namespace('https://schema.org/')
ex = Namespace('http://example.org/')

#Custom Schemas
ppl = Namespace('http://example.org/MovieDB/')
ppl2 = Namespace('http://example.org/MovieLens/')

net = Graph()


for index, row in integrated_ott_movies.iterrows():
    
    #assign types to movie
    net.add((URIRef(ppl+title_clean(row['title'])), RDF.type, SCHEMA['Movie']))
    
    #assigns the ott
    net.add((URIRef(ppl+title_clean(row['Source'])), RDF.type, ex['OTT']))
    net.add((URIRef(ppl+title_clean(row['title'])), ex['StreamingOn'], URIRef(ppl+title_clean(row['Source']))))
    
    #assigns the release year
    net.add((URIRef(ppl+title_clean(row['title'])), SCHEMA['datePublished'], Literal(str(row['release_year']), datatype=XSD['gYear'])))
    
    #assigns ratings
    if(str(row['rating'])!= 'nan'):
        net.add((URIRef(ppl+title_clean(row['title'])), RDF.type, SCHEMA['contentRating']))
        net.add((URIRef(ppl+title_clean(row['title'])), ex['IsRated'],URIRef(ppl+title_clean(row['rating']))))
    
    #links countries and the title
    if(str(row['country'])!= 'nan'):
        for item in row['country'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['countryOfOrigin']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['countryOfOrigin'],URIRef(ppl+title_clean(item))))
        
    
    if(str(row['listed_in'])!= 'nan'):
        #links genres from OTTs and the title
        for item in row['listed_in'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['genre']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['HasGenre'],URIRef(ppl+title_clean(item))))
        #links genres from MovieLens and the title    
        for item in row['genres'].split("|"):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl2+title_clean(item)), RDF.type, SCHEMA['genre']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['HasGenre'],URIRef(ppl2+title_clean(item))))    
            
            
    if(str(row['director'])!= 'nan'):
        for item in row['director'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['director']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['DirectedBy'],URIRef(ppl+title_clean(item))))     
            
            
    if(str(row['cast'])!= 'nan'):
        for item in row['cast'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['actor']))
            net.add((URIRef(ppl+title_clean(item)), ex['ActedIn'],URIRef(ppl+title_clean(row['title']))))        
                 
            
net.serialize(destination='OTT_KG.ttl',format='ttl')            
        



<Graph identifier=Na6221a5164454ea9be4b957bcb013cb4 (<class 'rdflib.graph.Graph'>)>

# Knowledge Graph 2 - Ratings data

In [11]:

#Existing Schemas
SCHEMA = Namespace('https://schema.org/')
ex = Namespace('http://example.org/')

#Custom Schemas
ppl = Namespace('http://example.org/MovieDB/')
ppl2 = Namespace('http://example.org/MovieLens/')

net = Graph()


for index, row in movie_review.iterrows():
    
    #assign types to movie
    net.add((URIRef(ppl2+str(int(row['movieId']))), ex['RatedBy'],URIRef(ppl2+title_clean("User_Id_" + str(int(row['userId']))))))
    

            
net.serialize(destination='ratings.ttl',format='ttl')            
        



<Graph identifier=Nbd36c7943dc2441d9da973e916f193e5 (<class 'rdflib.graph.Graph'>)>

# Knowledge Graph 3 - Integrated knowledge graph

In [12]:

#Existing Schemas
SCHEMA = Namespace('https://schema.org/')
ex = Namespace('http://example.org/')

#Custom Schemas
ppl = Namespace('http://example.org/MovieDB/')
ppl2 = Namespace('http://example.org/MovieLens/')

net = Graph()


for index, row in integrated_ott_movies.iterrows():
    
    #assign types to movie
    net.add((URIRef(ppl+title_clean(row['title'])), RDF.type, SCHEMA['Movie']))
    
    #links movielens id to the title
    net.add((URIRef(ppl+title_clean(row['title'])), OWL['sameAs'], URIRef(ppl2+title_clean(str(int(row['movieId']))))))
    
    #assigns the ott
    net.add((URIRef(ppl+title_clean(row['Source'])), RDF.type, ex['OTT']))
    net.add((URIRef(ppl+title_clean(row['title'])), ex['StreamingOn'], URIRef(ppl+title_clean(row['Source']))))
    
    #assigns the release year
    net.add((URIRef(ppl+title_clean(row['title'])), SCHEMA['datePublished'], Literal(str(row['release_year']), datatype=XSD['gYear'])))
    
    #assigns ratings
    if(str(row['rating'])!= 'nan'):
        net.add((URIRef(ppl+title_clean(row['title'])), RDF.type, SCHEMA['contentRating']))
        net.add((URIRef(ppl+title_clean(row['title'])), ex['IsRated'],URIRef(ppl+title_clean(row['rating']))))
    
    #links countries and the title
    if(str(row['country'])!= 'nan'):
        for item in row['country'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['countryOfOrigin']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['countryOfOrigin'],URIRef(ppl+title_clean(item))))
        
    
    if(str(row['listed_in'])!= 'nan'):
        #links genres from OTTs and the title
        for item in row['listed_in'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['genre']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['HasGenre'],URIRef(ppl+title_clean(item))))
        #links genres from MovieLens and the title    
        for item in row['genres'].split("|"):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl2+title_clean(item)), RDF.type, SCHEMA['genre']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['HasGenre'],URIRef(ppl2+title_clean(item))))    
            
            
    if(str(row['director'])!= 'nan'):
        for item in row['director'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['director']))
            net.add((URIRef(ppl+title_clean(row['title'])), ex['DirectedBy'],URIRef(ppl+title_clean(item))))     
            
            
    if(str(row['cast'])!= 'nan'):
        for item in row['cast'].split(","):
            item = item.replace("&","and")
            item = item.strip()
            
            net.add((URIRef(ppl+title_clean(item)), RDF.type, SCHEMA['actor']))
            net.add((URIRef(ppl+title_clean(item)), ex['ActedIn'],URIRef(ppl+title_clean(row['title']))))   
            
           
for index, row in movie_review.iterrows():
    
    #assign types to movie
    net.add((URIRef(ppl2+str(int(row['movieId']))), ex['RatedBy'],URIRef(ppl2+title_clean("User_Id_" + str(int(row['userId']))))))            
                 
            
net.serialize(destination='Integrated_KG.ttl',format='ttl')            
        



<Graph identifier=N9f3e0191381340f9968c79345a104471 (<class 'rdflib.graph.Graph'>)>

In [14]:
from tqdm import tqdm
net_df = pd.DataFrame()
for subject, predicate, object in tqdm(net.triples((None, None, None))):
    df = pd.DataFrame({'Subject':[subject],'Predicate':[predicate],'Object':[object]})
    net_df = pd.concat([net_df,df])
    

57233it [04:16, 222.70it/s]


In [24]:
net_df.reset_index(drop = True,inplace = True)

net_df['Subject'] = net_df['Subject'].apply(lambda x: str(x))
net_df['Predicate'] = net_df['Predicate'].apply(lambda x: str(x))
net_df['Object'] = net_df['Object'].apply(lambda x: str(x))

net_df.to_csv(r'Integrated_KG_CSV.csv',index= False)

"http://example.org/MovieDB/Pete's_Dragon"